Using the Earth Engine to build a MODIS fractional cover calibration
====================================================================
This is based on Scarth *etal* endmember extraction method.
The basic process is to:

 1. Assemble the field site data
 2. Extract the reflectannce data from the closest pixel in space and time
 3. Calculate the satellite viewed cover amount from the field data
 4. Compute the best fitting image endmembers based on the field and image data sets
 5. Use these endmembers to unmix the Earth Engine imagery

In [34]:

# Imports
from IPython.core.display import Image
import os, numpy, plotly, time, datetime
import ee


# Initialize the Earth Engine object, using your authentication credentials.
ee.Initialize()

# Initialise Plotly first time by running:
# python -c "import plotly; plotly.tools.set_credentials_file(username='xxxxxx', api_key='yyyyy')"

# Connect to the field site fusion table
fieldSites = ee.FeatureCollection('ft:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
# Have a look at the data structure using:
# print fieldSites.getInfo()['features'][1]

# Import MODIS Imagery
mcd43a4 = ee.ImageCollection('MODIS/MCD43A4')

# Get the latest image
latestImage = mcd43a4.sort('system:time_start', False ).limit(1).median()
# Have a look at the data structure using:
# print(latestImage.getInfo())
bufferedSites = fieldSites.map(lambda site:site.buffer(10000))
siteImage = latestImage.select('Nadir_Reflectance_Band6', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band1').paint(bufferedSites,10000)

#Get a bounding region for the thumbnail extract
coords = fieldSites.geometry().convexHull().buffer(200000).getInfo()['coordinates']
# Produce an image thumbnail
imageURL = siteImage.getThumbUrl({'region': coords,'format': 'png','min':100,'max': 5000,'dimensions': '800'})
print imageURL
# Display the image    
Image(url=imageURL)



https://earthengine.googleapis.com//api/thumb?thumbid=7e21f4bfe0fbcf6dea4e34c02dfb76d3&token=2bd1e9e0125af254b7db13793e5a6517


In [2]:

# Function to extract point data from imagery close in time
def pixelDataAtPoint(imageCollection,javascriptTimestamp,coordinates):
    # Convert to datetime object
    nominalDate = datetime.datetime.fromtimestamp(javascriptTimestamp/1000)
    # Buffer by 10 days
    sampleDaysRange = datetime.timedelta(days=10)
    # Start date 
    startDate = nominalDate - sampleDaysRange
    # End date
    endDate = nominalDate + sampleDaysRange
    # Select the relevant images
    extractCollection = ee.ImageCollection(imageCollection).filterDate(startDate, endDate);
    # Get the pixel value at a scale of 100m
    pointExtracted = extractCollection.getRegion(ee.Geometry.Point(coordinates[0],coordinates[1]),100)
    # create an image from the collection
    return pointExtracted.getInfo()

# Function that takes a getRegion list and computes the median of the reflectance data
def pixelListToMean(pointData):
    # List 0 is the band names
    #print pointData[0]
    pixelList = pointData[1:]
    # Convert to float
    pixelArray = numpy.ndarray.astype(numpy.array(pointData[1:])[:,4:],dtype=float)
    # Return the median
    try:
        arrayMedian = numpy.nanmedian(pixelArray,axis=0)
    except:
        arrayMedian = numpy.zeros_like(pixelArray[0])
    
    return arrayMedian


# Function to compute the fractional covers as viewed by the satellite for the site
# Required a site properties object
def fractionalCoverSatView(siteProperties):
    nTotal = siteProperties['num_points']
    # Canopy Layer
    nCanopyBranch = siteProperties['over_b'] * nTotal / 100.0
    nCanopyDead = siteProperties['over_d'] * nTotal / 100.0
    nCanopyGreen = siteProperties['over_g'] * nTotal / 100.0
    # Midstory Layer
    nMidBranch = siteProperties['mid_b'] * nTotal / 100.0
    nMidGreen = siteProperties['mid_g'] * nTotal / 100.0
    nMidDead = siteProperties['mid_d'] * nTotal / 100.0
    # Ground Layer
    nGroundDeadLitter = (siteProperties['dead'] + siteProperties['litter']) * nTotal / 100.0
    nGroundCrustDistRock = (siteProperties['crust'] + siteProperties['dist'] + siteProperties['rock']) * nTotal / 100.0
    nGroundGreen = siteProperties['green'] * nTotal / 100.0
    nGroundCrypto = siteProperties['crypto'] * nTotal / 100.0
    # Work out the canopy elements as viewed from above
    canopyFoliageProjectiveCover = nCanopyGreen / (nTotal - nCanopyBranch)
    canopyDeadProjectiveCover = nCanopyDead / (nTotal - nCanopyBranch)
    canopyBranchProjectiveCover = nCanopyBranch / nTotal * (1.0 - canopyFoliageProjectiveCover - canopyDeadProjectiveCover)
    canopyPlantProjectiveCover = (nCanopyGreen+nCanopyDead + nCanopyBranch) / nTotal
    # Work out the midstorey fractions
    midFoliageProjectiveCover = nMidGreen / nTotal
    midDeadProjectiveCover = nMidDead / nTotal
    midBranchProjectiveCover = nMidBranch / nTotal
    midPlantProjectiveCover = (nMidGreen + nMidDead + nMidBranch) / nTotal
    # Work out the midstorey  elements as viewed by the satellite using a gap fraction method
    satMidFoliageProjectiveCover = midFoliageProjectiveCover * (1 - canopyPlantProjectiveCover)
    satMidDeadProjectiveCover = midDeadProjectiveCover * (1 - canopyPlantProjectiveCover)
    satMidBranchProjectiveCover = midBranchProjectiveCover * (1 - canopyPlantProjectiveCover)
    satMidPlantProjectiveCover = midPlantProjectiveCover * (1 - canopyPlantProjectiveCover)
    # Work out the groundcover fractions as seen by the observer
    groundPVCover = nGroundGreen / nTotal
    groundNPVCover = nGroundDeadLitter / nTotal
    groundBareCover = nGroundCrustDistRock / nTotal
    groundCryptoCover = nGroundCrypto / nTotal
    groundTotalCover = (nGroundGreen + nGroundDeadLitter + nGroundCrustDistRock) / nTotal
    # Work out the ground cover propoetions as seen by the satellite
    satGroundPVCover = groundPVCover * (1 - midPlantProjectiveCover) * (1 - canopyPlantProjectiveCover)
    satGroundNPVCover = groundNPVCover * ( 1- midPlantProjectiveCover) * (1 - canopyPlantProjectiveCover)
    satGroundBareCover = groundBareCover * (1 - midPlantProjectiveCover) * (1 - canopyPlantProjectiveCover)
    satGroundCryptoCover = groundCryptoCover * (1 - midPlantProjectiveCover) * (1 - canopyPlantProjectiveCover)
    satGroundTotalCover = groundTotalCover * (1 - midPlantProjectiveCover) * (1 - canopyPlantProjectiveCover)
    # Final total covers calculated using gap probabilities through all layers
    totalPVCover = canopyFoliageProjectiveCover + satMidFoliageProjectiveCover + satGroundPVCover
    totalNPVCover = canopyDeadProjectiveCover + canopyBranchProjectiveCover + satMidDeadProjectiveCover + satMidBranchProjectiveCover + satGroundNPVCover
    totalBareCover = satGroundBareCover
    totalCryptoCover = satGroundCryptoCover
    
    return numpy.array([totalPVCover,totalNPVCover,totalBareCover,totalCryptoCover])
    

   
# Create an empty array
fractionalCoverArray = numpy.array([])
modisReflectanceArray = numpy.array([])

# Extract the data for all the sites
for site in fieldSites.getInfo()['features']:
    #print site['properties']['site']
    
    
    # Extract the reflectance data from the nearest images
    coordinates = site['geometry']['coordinates']
    obsTime = site['properties']['obs_time']
    # Check if the data is within the MODIS era
    if datetime.datetime.fromtimestamp(obsTime/1000)>datetime.datetime(2000, 1, 1, 0, 0):
        
        try:
            # Wait for a bit to avoid hitting the EE quota limits
            time.sleep(1.0)
            sitePixelList =  pixelDataAtPoint(mcd43a4,obsTime,coordinates)
        except:
            # Try again after a longer break
            time.sleep(60.0)
            sitePixelList =  pixelDataAtPoint(mcd43a4,obsTime,coordinates)

        meanReflectance = pixelListToMean(sitePixelList)
        # Data Array contains the 7 reflectance bands
        modisReflectanceArray = numpy.append(modisReflectanceArray,meanReflectance)


        # Calculate the fractional cover as seen by the satellite
        siteCover = fractionalCoverSatView(site['properties'])
        # Data Array contains the 4 fractional cover values
        fractionalCoverArray = numpy.append(fractionalCoverArray,siteCover)
    
    


WA_Gas_030
WA_Gas_029
WA_Gas_031
WA_Gas_035
WA_Gas_034
WA_Gas_038b
WA_Gas_036
WA_Gas_033
WA_Gas_032
WA_Gas_038a
WA_Gas_038
WA_Gas_037
Agr_WA_039
Agr_WA_041
Agr_WA_040
WA_Agr_058
WA_Agr_057
Agr_WA_043
Agr_WA_044A
Agr_WA_042
Agr_WA_044
WA_Yalgoo09
WA_Yalgoo08
WA_Yalgoo07
WA_Yalgoo06
WA_Yalgoo05
WA_Yalgoo04
WA_Pilb_018
WA_Pilb_017
WA_Yalgoo02
WA_Yalgoo03
WA_E_Gas_125
WA_E_Gas_124
WA_Pilb_021
WA_Pilb_020
WA_Pilb_019
WA_Pilb_022
WA_Agr_056
WA_Agr_055
WA_Murc_099
WA_Agr_054
WA_Murc_098
WA_Agr_051
WA_Agr_052
WA_Agr_053
WA_Murc_097
WA_Murc_096
WA_Murc_095
WA_Yalgoo01
WA_Murc_094
WA_E_Gas_123
WA_Agr_059
WA_Murc_122
WA003
WA002
WA_Agr_060
WA001
WA_Murc_121
WA_E_Gas_120
WA_E_Gas_119
WA_E_Gas_118
WA_E_Gas_117
WA_E_Gas_116
WA_Murc_100
WA_Murc_101
WA_Murc_104
WA_Murc_103
WA_Murc_102
WA_Pilb_028a
WA_Pilb_024
WA_Pilb_023
WA_Pilb_025
WA_E_Pilb_109
WA_E_Pilb_110
WA_Pilb_028
WA_E_Pilb_111
WA_E_Pilb_112
WA_E_Pilb_113
WA_E_Pilb_115
WA_E_Pilb_114
WA_N_Gold_132
WA_Murc_108
WA_N_Gold_133
WA_Murc_107
WA_N_Gold

/home/ubuntu/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:739: RuntimeWarning:

All-NaN slice encountered



In [3]:
# Reshape the arrays based on the 7 reflectance or 4 cover parameters
modisReflectanceArray = numpy.reshape(modisReflectanceArray,(-1,7))
fractionalCoverArray = numpy.reshape(fractionalCoverArray,(-1,4))
# Remove NANs
isGoodIDX = numpy.where(~numpy.isnan(xdata))
modisReflectanceArray = modisReflectanceArray[isGoodIDX]
fractionalCoverArray = fractionalCoverArray[isGoodIDX]
# And save the extracted data for reuse
numpy.savetxt('modisReflectanceArray.txt', modisReflectanceArray)
numpy.savetxt('fractionalCoverArray.txt', fractionalCoverArray)

Explore the data
----------------

Now we've extracted from the earth engine, we can undertake some basic data analysis

In [62]:
# Load the data in from this point to continue
modisReflectanceArray = numpy.loadtxt('modisReflectanceArray.txt')
fractionalCoverArray = numpy.loadtxt('fractionalCoverArray.txt')

The traditional red band vs cover relationship
----------------
As used in arid environments - works best locally

In [71]:
from scipy import stats
# Fit some data
xdata = modisReflectanceArray[:,0] # Red band
ydata = fractionalCoverArray[:,2] # Bare ground

# Polynomial Fitting
slope, intercept, r_value, p_value, std_err = stats.linregress(xdata,ydata)
print "r squared = %f" % r_value**2

fitPolyFun = numpy.poly1d([slope,intercept])
fitData = fitPolyFun(xdata)


# Create a site trace
traceSites = plotly.graph_objs.Scatter(
    x = xdata,
    y = ydata,
    mode = 'markers',
    name = 'Sites',
    marker=dict(color='red',symbol='circle',opacity=0.5)
)

# Create a fit trace
traceFit = plotly.graph_objs.Scatter(
    x = xdata,
    y = fitData,
    mode = 'lines',
    name = 'Fitted Line',
    line = dict(color = ('rgb(22, 96, 167)'),width = 4)
)

data = [traceSites,traceFit]

layout = plotly.graph_objs.Layout(
    title='Modis Red Band vs Bare Ground',
    xaxis=dict(
        title='MODIS Reflectance',
        ticklen=5,
        zeroline=False,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Bare Ground',
        ticklen=5,
        gridwidth=2,
    ),
)

plotFig = plotly.graph_objs.Figure(data=data, layout=layout)

# Plot and embed in ipython notebook!
plotly.plotly.iplot(plotFig, filename='basic-scatter')

r squared = 0.322835


Check out the range of cover values
----------------
Using a histogram of bare ground over all the sites

In [73]:
data = [plotly.graph_objs.Histogram(x=fractionalCoverArray.sum(axis=1))]
data = [plotly.graph_objs.Histogram(x=fractionalCoverArray[:,2])]
plotly.plotly.iplot(data, filename='fractionalSums-histogram')